In [ ]:
import collections
import datetime
import time

In [ ]:
import numpy as np

In [ ]:
from bokeh.plotting import figure, show, output_notebook, output_server, curdoc
from bokeh.client import push_session

*To run these examples you must execute the command `bokeh serve --allow-websocket-origin=localhost:8888` in the top-level Bokeh source directory first.*

In [ ]:
# open a session to keep our local document in sync with server
session = push_session(curdoc(), session_id='default')

In [ ]:
# first call output_server so the notebook cells are loaded from the configured server
output_server()

# then call output_notebook to generate output in Jupyter notebook cells when show is called
output_notebook()

In [ ]:
TS_MULT_us = 1e6
UNIX_EPOCH = datetime.datetime(1970, 1, 1, 0, 0) #offset-naive datetime

def int2dt(ts, ts_mult=TS_MULT_us):
    """Convert timestamp (integer) to datetime"""
    return(datetime.datetime.utcfromtimestamp(float(ts)/ts_mult))
        
def td2int(td, ts_mult=TS_MULT_us):
    """Convert timedelta to integer"""
    return(int(td.total_seconds()*ts_mult))
        
def dt2int(dt, ts_mult=TS_MULT_us):
    """Convert datetime to integer"""
    delta = dt - UNIX_EPOCH
    return(int(delta.total_seconds()*ts_mult))
        
def int_from_last_sample(dt, td):
    return(dt2int(dt) - dt2int(dt) % td2int(td))

In [ ]:
TS_MULT = 1e3
td_delay = datetime.timedelta(seconds=0.1)
delay_s = td_delay.total_seconds()
delay_int = td2int(td_delay, TS_MULT)

value = 1000 # initial value
N = 100 # number of elements into circular buffer

buff = collections.deque([value]*N, maxlen=N)

In [ ]:
t_now = datetime.datetime.utcnow()
ts_now = dt2int(t_now, TS_MULT)
t = collections.deque(np.arange(ts_now-N*delay_int, ts_now, delay_int), maxlen=N)

In [ ]:
p = figure(x_axis_type="datetime")
p.x_range.bounds = p.y_range.bounds = None
r = p.line(t, buff, color="#0000FF", name="line_example")

In [ ]:
show(p)

In [ ]:
ds = r.data_source

def update():
    ts_now = dt2int(datetime.datetime.utcnow(), 1e3)
    t.append(ts_now)

    value = ds.data['y'][-1] + np.random.uniform(-1, 1)
    buff.append(value)
    
    ds.data.update(x=list(t), y=list(buff))
    
curdoc().add_periodic_callback(update, delay_s*1000)
session.loop_until_closed()